## ATTENTION-BI-LSTM TO-GO

In [1]:
# 각 파일에 맞게 아래 두 경로만 재설정해주세요.
from google.colab import drive
drive.mount('/content/drive')
TOKENPATH = '/content/drive/MyDrive/finalvoice/bi_lstm_attention_tokenizer.pickle'
MODELPATH = "/content/drive/MyDrive/finalvoice/bi_lstm_attention_model.h5"

# ------<이 아래로는 수정할 필요 없습니다>------
# 커스텀레이어 정의
from tensorflow.keras.layers import Layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        # Attention 가중치를 위한 weight 생성
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)
    def call(self, x):
        # Attention score 계산
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
# 필요 모듈, tokenizer, 모델 불러오기
!pip install konlpy
from konlpy.tag import Okt
okt = Okt()
import pickle
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/finalvoice/bi_lstm_attention_model.h5", custom_objects={'AttentionLayer': AttentionLayer})
with open('/content/drive/MyDrive/finalvoice/bi_lstm_attention_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# 확률연산
def predict_label_prob(sentence):
    # 값 초기화
    #predicted_prob = 0
    # 문장 토큰화
    tokenized_sentence = okt.morphs(sentence)
    # 토큰화된 문장을 숫자 시퀀스로 변환
    sequence = tokenizer.texts_to_sequences([tokenized_sentence])
    # 패딩 처리
    MAX_LENGTH = 100
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LENGTH)
    # 예측 수행
    prediction = model.predict(padded_sequence)
    # 예측 확률 반환
    return prediction[0][0]

while True:
    user_input = input("문장을 입력하세요: ")
    # 사용자가 공백을 입력하면 종료
    if not user_input.strip():
        break
    predicted_prob = predict_label_prob(user_input)
    print(f"라벨 0의 확률: {1 - predicted_prob:.4f}")
    print(f"라벨 1의 확률: {predicted_prob:.4f}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 49.3 MB/s eta 0:00:00
문장을 입력하세요: ㅇㅇ
1/1 [==============================] - 5s 5s/step
라벨 0의 확률: 0.1062
라벨 1의 확률: 0.8938
문장을 입력하세요: 수고하세요.
1/1 [==============================] - 0s 20ms/step
라벨 0의 확률: 0.1073
라벨 1의 확률: 0.8927
문장을 입력하세요: 감자탕 집 아들
1/1 [==============================] - 0s 20ms/step
라벨 0의 확률: 0.1181
라벨 1의 확률: 0.8819
문장을 입력하세요: 뭐야 왜 이렇게 높게 나와 ㅠㅠㅠㅠㅠ
1/1 [==============================] - 0s 21ms/step
라벨 0의 확률: 0.1070
라벨 1의 확률: 0.8930
문장을 입력하세요: 아니 왜 다 9
1/1 [==============================] - 0s 20ms/step
라벨 0의 확률: 0.1286
라벨 1의 확률: 0.8714
문장을 입력하세요: 멍청이야?
1/1 [==============================] - 0s 21ms/step
라벨 0의 확률: 0.1073
라벨 1의 확률: 0.8927
문장을 입력하세요: 통장 신분증 제출
1/1 [==============================] - 0s 20ms/step
라벨 0의 확률: 0.1091
라벨 1의 확률: 0.8909
문장을 입력하세요: 강제 검찰 통장 유포 대포 정치 마약 사범 공작 공항 활용 저장
1/1 [==============================] - 0s 20ms/st

KeyboardInterrupt: ignored